In [1]:
from qat.lang.AQASM import Program, QRoutine, RZ, RX, CNOT, H, X, Z
from qat.core import Observable, Term
from qat.qpus import get_default_qpu, PyLinalg
from qat.plugins import ScipyMinimizePlugin
from copy import deepcopy
import matplotlib.pyplot as plt
import math as m

# Hands-on: Algoritmo de Grover

## Sumário do Algoritmo de Grover

Seja $N=2^n$. Consideramos um registrador de $n$ qubits de modo que um dos elementos da base computacional é dado por $|x_0\rangle$. Definimos inicialmente uma condição inicial uniforme

$$|\psi_0\rangle =  \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1} |x\rangle$$

Daí, definimos os operadores:

$$U_{f} |x\rangle|i\rangle=|x\rangle|i\oplus f(x)\rangle,$$
chamado de oráculo (codifica a função $f$ onde $f(x_0)=-1$ e $f(x)=0$ se $x\neq x_0$, onde $x_0$ é chamado elemento marcado).
O operador $G$ é
$$G = (2|\psi_0\rangle\langle \psi_0| - \mathrm{I}_N)\otimes \mathrm{I}_2, $$
chamado de operador de inversão em torno da média.

## Os passos do algoritmo

O algoritmo tem os seguintes passos (usando $n+1$ qubits)

1. Primeiro, preparar o estado inicial $|\psi_0\rangle|-\rangle$;

2. Aplicar o operador $U_f$;

3. Aplicar o operador $G$;

4. Repetir $t$ vezes os passos 2 e 3 onde

$$
t=\left\lfloor\frac{\pi}{4}\sqrt N\right\rfloor;
$$

5. Medir o primeiro registrador na base computacional.

A medição retorna o estado $x_0$ com probabilidade de aproximadamente $1 - \frac{1}{\sqrt{N}}$.

### Execício 1

(a) Implemente no $\verb|QLM|$ o circuito do operador $U_{f}$ no caso $N=4$ e $x_0=10$, ou seja, $f(10)=1$ e $f(j)=0$ se $ j\neq 10$. Mostre o circuito.

In [ ]:
n = #FIXME
oraculo = QRoutine()

wires = oraculo.new_wires(n + 1) # crie 3 fios

X(wires[#FIXME])
X.ctrl(n)(wires) # Toffoli multiqubit
X(wires[#FIXME])

%qatdisplay oraculo --svg

(b) Rode o circuito quando a entrada é $|10\rangle|0\rangle$ e verifique que a saída é $|10\rangle|1\rangle$, como previsto.

In [ ]:
qprog = Program()
Qr = qprog.qalloc(n+1)
Cr = qprog.calloc(n+1)

# FIXME  ### aplique X no primeiro qubit para gerar |10>|0>

# FIXME  ### aplique o oraculo nos qubits ### solução: oraculo(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

%qatdisplay circuit --svg

# Create a Quantum Processor Unit
linalgqpu = PyLinalg()

# Create a job
job = circuit.to_job()

# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("state",sample.state,"amplitude",sample.amplitude) 

(c) Rode o circuito quando a entrada é $|10\rangle|-\rangle$ e verifique que a saída é $\big(-|10\rangle|-\rangle\big)$, como previsto.

(d) Rode o circuito quando a entrada é $|\psi_0\rangle|0\rangle$ onde
$$|\psi_0\rangle =  \frac{1}{2}\sum_{i=0}^{3} |x\rangle$$
e verifique que a saída é uma superposição que tem $|1\rangle$ no segundo registrador apenas para o elemento marcado, como previsto.

(e) Rode o circuito quando a entrada é $|\psi_0\rangle|-\rangle$ e verifique que a saída é uma superposição com sinal invertido apenas para o elemento marcado, como previsto.

### Exercício 2

O objetivo deste exercício é aprender implementar o operador
$$ A=( \mathrm{I}_N - 2|0\rangle\langle 0| ) {\otimes }\mathrm{I}_2$$
quando a entrada do segundo registrador é $|-\rangle$ (a sua implementação só funciona neste caso).

(a) Verifique através de uma análise algébrica que $A=U_f$, onde $f(0)=1$ e $f(j)=0$ caso contrário (sempre tome a entrada do segundo registrado como $|-\rangle$).

(b) Implemente no $\verb|QLM|$ o circuito do operador $A$ para $N=4$.  Rode o circuito quando a entrada é $|\psi_0\rangle|-\rangle$ onde
$$|\psi_0\rangle =  \frac{1}{2}\sum_{i=0}^{3} |x\rangle$$
e verifique que a saída é uma superposição com sinal invertido apenas quando o primeiro registrador é $|0\rangle$, como previsto.

### Exercício 3

O objetivo deste exercício é aprender implementar o operador
$$ G=( \mathrm{I}_N - 2|\psi_0\rangle\langle\psi_0| ) {\otimes }\mathrm{I}_2$$
quando a entrada do segundo registrador é $|-\rangle$ (a sua implementação só funciona neste caso). Vimos na aula que $G=H^{\otimes n} A H^{\otimes n}$. Use este fato e a implementação de $A$ vista acima para fazer este exercício. [*NOTA: O operador $G$ neste exercício tem uma inversão de sinal. Isto não afeta o resultado do algoritmo, como visto em aula.*]

(a) Implemente no $\verb|QLM|$ o circuito do operador $G$ para $N=4$.  Rode o circuito quando a entrada é $|\psi_0\rangle|-\rangle$ onde
$$|\psi_0\rangle =  \frac{1}{2}\sum_{i=0}^{3} |x\rangle$$
e verifique que a saída é igual a $(-|\psi_0\rangle|-\rangle)$, como previsto.

(b) Escolha um vetor ortogonal a $|\psi_0\rangle$. Vamos denotar por $|\psi_0^\perp\rangle$. Rode o circuito quando a entrada é $|\psi_0^\perp\rangle|-\rangle$
e verifique que a saída é igual a entrada, como previsto.


### Exercício 4

Determine o número de repetições do algoritmo de Grover quando $N=4$.

### Exercício 5

Implemente o algoritmo de Grover completo para $N=4$ e rode o algoritmo quando a entrada é $|0\rangle|-\rangle$ e o elemento marcado é $x_0=2$. Verifique se a saída da medição do primeiro registrador tem como saída a cadeia de bits correta.

## Algoritmo de Grover com mais de 1 elemento marcado

### Execício 6

(a) Implemente no $\verb|QLM|$ o circuito do operador $U_{f}$ no caso $N=8$ tal que $f(x)=1$ se $x=110$ ou $x=011$ e $f(x)=0$ se caso contrário. Mostre o circuito.

(b) Rode o circuito quando a entrada é $|\psi_0\rangle|0\rangle$ onde
$$|\psi_0\rangle =  \frac{1}{\sqrt{8}}\sum_{i=0}^{7} |x\rangle$$
e verifique que a saída é uma superposição que tem $|1\rangle$ no segundo registrador apenas para os elementos marcados, como previsto.

(c) Rode o circuito quando a entrada é $|\psi_0\rangle|-\rangle$ e verifique que a saída é uma superposição com sinal invertido apenas para os elementos marcados, como previsto.

### Exercício 7

Seja
$$ G=( \mathrm{I}_N - 2|\psi_0\rangle\langle\psi_0| ) {\otimes }\mathrm{I}_2$$
Tome $N=8$ e seja
$$|\psi_0\rangle =  \frac{1}{\sqrt{8}}\sum_{i=0}^{7} |x\rangle$$

(a) Implemente no $\verb|QLM|$ o circuito do operador $G$ para $N=8$.  Rode o circuito quando a entrada é $|\psi_0\rangle|-\rangle$ e verifique que a saída é igual a $(-|\psi_0\rangle|-\rangle)$, como previsto.

(b) Escolha um vetor ortogonal a $|\psi_0\rangle$. Vamos denotar por $|\psi_0^\perp\rangle$. Rode o circuito quando a entrada é $|\psi_0^\perp\rangle|-\rangle$
e verifique que a saída é igual a entrada, como previsto.

### Exercício 8

Determine o número de repetições do algoritmo de Grover quando $N=8$ com $m=2$ elementos marcados. Use a fórmula $\left\lfloor\frac{\pi}{4}\sqrt{\frac{N}{m}}\right\rfloor$.

### Exercício 9

Implemente o algoritmo de Grover completo para $N=8$ com os elementos marcados $110$ e $011$. Rode o algoritmo quando a entrada é $|0\rangle|-\rangle$ e o elemento marcado é $x_0=2$. Verifique se a saída da medição do primeiro registrador é a cadeia de bits correta.

## Implementação econômica do algoritmo de Grover

### Execício 10

O objetivo do próximo exercício é implementar o algoritmo de Grover sem usar o segundo registrador. Vamos re-definir o operador $U_f$ usando apenas o primeiro registrador ($n$ qubits) como
$$U_{f} |x\rangle=(-1)^{f(x)}|x\rangle,$$
onde $f(x_0)=1$ e $f(x)=0$ se $x\neq x_0$ e $x_0=(i_1,...,i_n)_2$. Use o seguinte circuito para implementar $U_f$:

<p align="center">
<img src="imagens/Grover-econ.png" width="450px">
</p>

Note que $X^{i_1}=\text{I}$ se $i_1=0$ e $X^{i_1}=X$ se $i_1=1$.

(a) Implemente no $\verb|QLM|$ o circuito do operador $U_{f}$ no caso $N=8$ tal que $f(x)=1$ se $x=110$ e $f(x)=0$ caso contrário. Mostre o circuito.

(b) Rode o circuito quando a entrada é $|\psi_0\rangle$ onde
$$|\psi_0\rangle =  \frac{1}{\sqrt{8}}\sum_{i=0}^{7} |x\rangle$$
e verifique que a saída é uma superposição com sinal invertido apenas para o elemento marcado, como previsto.

(c) Implemente no $\verb|QLM|$ o circuito do operador $G$ no caso $N=8$ usando apenas 3 qubits. Use o fato de que $G=H^{\otimes n}U_{f_0}H^{\otimes n}$ quando $f_0(x)=1$ se $x=000$ e $f_0(x)=0$ se caso contrário. Mostre o circuito e cheque que o operador $G$ está implementado corretamente.

(d) Implemente no $\verb|QLM|$ o circuito do algoritmo de Grover no caso $N=8$ com $x_0=110$ usando apenas 3 qubits. Faça um histograma que mostre a probabilidade de sucesso com pelo menos 3 casas decimais. Compare com o valor exato de $121/128$.